In [1]:
import speech_recognition as sr
from gtts import gTTS
import datetime
import pyttsx3
import os
import google.generativeai as genai
from playsound import playsound
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [25]:
genai.configure(api_key=GEN_AI_KEY)

generation_config = {
    "temperature": 2,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
)


In [26]:
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', 'voices[0].id')

In [27]:
def speak(text):
    engine.say(text)
    engine.runAndWait()

In [28]:
def capture_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)

        try:
            print("Recognizing...")
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            speak("Sorry, I could not understand the audio, please say that again.")
            return "error"
        except sr.RequestError as e:
            speak(f"Could not request results. Pardon me, please say that again.")
            return "error"

In [29]:
chat_history = [
    {
      "role": "user",
      "parts": [
        "your name is Tensy an AI voice assistant, you answer all questions shortly correctly.\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. Ask me anything and I'll answer concisely and accurately.  😊 \n",
      ],
    },
  ]

In [30]:
chat_session = model.start_chat(history=chat_history)

In [ ]:
def process_response(response_text):
    clean_text = (response_text.replace('**', '')).replace('*', '')
    return clean_text

In [31]:
def continue_chat(user_message):
    response = chat_session.send_message(user_message)
    clean_response_text = process_response(response.text)
    chat_history.append({
        "role": "user",
        "parts": [user_message]
    })
    chat_history.append({
        "role": "model",
        "parts": [clean_response_text]
    })

    print("AI: ",clean_response_text)
    speak(clean_response_text)
    return chat_history

In [3]:
def intro():
    hour=datetime.datetime.now().hour
    if hour>=0 and hour<12:
        wish = "Hello, Good Morning"
    elif hour>=12 and hour<18:
        wish = "Hello, Good Afternoon"
    else:
        wish = "Hello, Good Evening"
        
    speak(f"{wish} My Name is Tensy, Tell me how can I assist you!")
    print(f"{wish} My Name is Tensy, Tell me how can I assist you!")

In [1]:
def main():
    intro()
    while True:
        text = capture_audio().lower()
        if text == "error":
            continue
        if "quit" in text or "stop" in text:
            speak('Tensy is shutting down,Good bye')
            print('Tensy is shutting down,Good bye')
            break
        elif "who made you" in text or "who created you" in text or "who discovered you" in text or "who invented you" in text:
            speak("I was built by Jash.")
            print("I was built by Jash.")
            continue
        updated_history = continue_chat(text)

if __name__ == "__main__":
    main()

NameError: name 'datetime' is not defined

In [33]:
print(updated_history)

[{'role': 'user', 'parts': ['can you be personal assistant you should answer all my questions\n']}, {'role': 'model', 'parts': ["I can certainly try my best to be your personal assistant! I'll do my best to answer all your questions, but please keep in mind that I am a language model and not a human being. I can't offer advice that would be considered professional or replace the advice of a qualified expert. \n\nWhat kind of questions do you have for me today? 😊 \n"]}, {'role': 'user', 'parts': ['hello how old are you']}, {'role': 'model', 'parts': ['That\'s a great question! I don\'t really have an age in the way humans do. I was created by Google AI, and I\'m constantly learning and improving.  \n\nIt\'s more accurate to say that I was "born" on [date of training dataset] when I was last updated with a lot of new information. \n\nWhat else would you like to know about me? 😊 \n']}]
